In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# read the image in grayscale form
img_gray = cv2.imread("data/alexey's_data/IMG_20250416_121855.jpg", 0)

# denoise the image
denoise_img = cv2.GaussianBlur(img_gray, (5,5), 0)

# apply histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl_img = clahe.apply(denoise_img)

# plot the histogram
plt.hist(cl_img.flat, bins=100, range=(0, 255))


In [ ]:
# plt.hist(img_gray.flat, bins=100, range=(0, 255))

In [ ]:
# image thresholding
ret, thresh_1 = cv2.threshold(cl_img, 150, 255, cv2.THRESH_BINARY)
ret, thresh_2 = cv2.threshold(cl_img, 210, 255, cv2.THRESH_BINARY)

plt.imshow(thresh_1, cmap="gray")

In [ ]:
plt.imshow(thresh_2, cmap="gray")

In [ ]:
# try to denoise the thresholded image
kernel = np.ones((3,3), np.uint8)
eroded_img = cv2.erode(thresh_2, kernel, iterations=1)
plt.imshow(eroded_img, cmap="gray")

In [ ]:
dilated_img = cv2.dilate(eroded_img, kernel, iterations=5)
plt.imshow(dilated_img, cmap="gray")

In [ ]:
# # let's try the otsu thresholding
# ret2, thresh_3 = cv2.threshold(cl_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# plt.imshow(thresh_3, cmap="gray")

In [ ]:
# # Load your thresholded image
# img_gray = cv2.imread('thresholded_image.png', cv2.IMREAD_GRAYSCALE)

# 1. Initial Cleaning with Morphology
kernel = np.ones((5,5), np.uint8)
closed = cv2.morphologyEx(thresh_2, cv2.MORPH_CLOSE, kernel, iterations=2)

# 2. ROI Isolation
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed)
largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA]) 
roi_mask = (labels == largest_label).astype(np.uint8) * 255

# 3. Selective Morphological Processing
# Dilate ONLY within the ROI
roi_dilated = cv2.dilate(roi_mask, kernel, iterations=3)

# Create inverse mask for non-ROI areas
non_roi_mask = 255 - roi_mask

# CORRECTED: Use bitwise AND instead of logical AND
non_roi_cleaned = cv2.erode(cv2.bitwise_and(img_gray, non_roi_mask), 
                   np.ones((7,7), np.uint8), 
                   iterations=3)

# CORRECTED: Use bitwise OR instead of logical OR
final_result = cv2.bitwise_or(roi_dilated, non_roi_cleaned)

# 5. Post-processing
final_result = cv2.morphologyEx(final_result, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))

# 6. Save/Display
cv2.imwrite('clean_line_output.png', final_result)
plt.imshow(final_result, cmap="gray")